# Ingest session attendance
https://docs.wonde.com/docs/api/sync#get-session-attendance


In [6]:
%run _Config

StatementMeta(, 2df91136-15f7-44f5-9ce8-e82c23a45b51, -1, Finished, Available)

In [7]:
%run _WondeStructs

StatementMeta(, 2df91136-15f7-44f5-9ce8-e82c23a45b51, -1, Finished, Available)

In [8]:
%run _WondeAPI

StatementMeta(, 2df91136-15f7-44f5-9ce8-e82c23a45b51, -1, Finished, Available)

In [9]:
#Select timestamp for watermarks.
currentUTC = datetime.utcnow()
strCurrentUTC = currentUTC.strftime('%Y-%m-%d %H:%M:%S')

StatementMeta(, 2df91136-15f7-44f5-9ce8-e82c23a45b51, 31, Finished, Available)

In [10]:
# Get basic json return
#url = "https://api.wonde.com/v1.0/schools/A1128526306/attendance-leavers/session?updated_after=2023-07-16 09:00:00"
#session = requests.Session()
#data = session.get(url, headers=wondeHeaders).json()


#print(data)

StatementMeta(, 2df91136-15f7-44f5-9ce8-e82c23a45b51, 32, Finished, Available)

### Get list of available schools with attendance watermarks

In [11]:
df = (spark.read
                .format("delta")
                .load(f"{BronzeBasePath}/Watermarks")
    )

dfSchoolIds = df.filter(df["session_attendance_last_update"].isNotNull())
display(dfSchoolIds)

StatementMeta(, 2df91136-15f7-44f5-9ce8-e82c23a45b51, 33, Finished, Available)

SynapseWidget(Synapse.DataFrame, 8a03ee11-718f-4bf2-a943-6e8540efbfaa)

### Functions to retrieve data for given school.

In [12]:
def process_attendance_data(url, school_id, watermark_time):
    df = getWondeAPI(url, wondeHeaders, session_attendance_schema)
    df = df.withColumn("school_id", lit(school_id))

    df_final = (df
        .select(
            col("school_id"),
            col("id").alias("attendance_mark_id"),
            to_date(col("date.date"), "yyyy-MM-dd HH:mm:ss.SSSSSS").alias("date"),
            col("session"),
            col("comment"),
            col("employee").alias("employee_id"),
            col("attendance_code").alias("session_attendance_code_id"),
            col("student").alias("student_id"),
        )
        .withColumn("extracted_at", lit(watermark_time))
        .withColumn("extracted_year", year(lit(watermark_time)))
        .withColumn("extracted_month", month(lit(watermark_time)))
        .withColumn("extracted_day", dayofmonth(lit(watermark_time)))
    )

    (df_final.write
        .mode("append")
        .format("delta")
        .partitionBy("school_id", "extracted_year", "extracted_month", "extracted_day")
        .save(f"{BronzeBasePath}/SessionAttendance")
    )

def getUpdatedAttendance(strSchoolId, strUpdatedAfter):

    # On Roll Students
    url = f"{baseWondeUrl}/schools/{strSchoolId}/attendance/session?updated_after={strUpdatedAfter}"
    process_attendance_data(url, strSchoolId, currentUTC)

    # Leaver Students
    url = f"{baseWondeUrl}/schools/{strSchoolId}/attendance-leavers/session?updated_after={strUpdatedAfter}"
    process_attendance_data(url, strSchoolId, currentUTC)

    spark.sql(f"UPDATE Bronze.Watermarks SET session_attendance_last_update = '{currentUTC}' WHERE school_id = '{strSchoolId}'")


StatementMeta(, 2df91136-15f7-44f5-9ce8-e82c23a45b51, 34, Finished, Available)

In [13]:
for school in dfSchoolIds.collect():
    schoolId = school.school_id
    strUpdatedTime = school.session_attendance_last_update.strftime('%Y-%m-%d %H:%M:%S')
    getUpdatedAttendance(schoolId, strUpdatedTime)
    print(schoolId)
    print(strUpdatedTime)

StatementMeta(, 2df91136-15f7-44f5-9ce8-e82c23a45b51, 35, Submitted, Running)

A1930499544
2022-08-31 08:00:00
A595075426
2022-08-31 08:00:00
A787417329
2022-08-31 08:00:00
A1998103997
2022-08-31 08:00:00
A1130851934
2022-08-31 08:00:00
A220166484
2022-08-31 08:00:00
A949960374
2022-08-31 08:00:00
